In [260]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr

In [261]:
Predicted = pd.read_csv("./Seurat/Outputs/output_exp1.csv")
Target=pd.read_csv("./Data/exp1/target.csv")

In [262]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
Predicted.drop(columns="prediction.score.max", inplace=True)


In [263]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [264]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [265]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [266]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [267]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [268]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [269]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [270]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [271]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

/opt/homebrew/Caskroom/miniforge/base/envs/ML/lib/python3.10/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/homebrew/Caskroom/miniforge/base/envs/ML/lib/python3.10/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [272]:
np.mean(np.nan_to_num(d))

0.3089027479942252

In [273]:
d

array([-0.06453909,  0.69827437,  0.60978776, -0.02214815,  0.19475775,
        0.6316529 ,  0.092623  ,  0.03092159,  0.25783079,  0.64926831,
        0.59513742,  0.81003434,         nan,  0.34860956, -0.08437014,
        0.68827929,  0.57056947, -0.00760445, -0.06267167,  0.34494612,
        0.01109006,  0.7280833 ,  0.64928819,  0.30028232,  0.87086028,
        0.02918347,  0.28730712,  0.23564437,  0.01887359, -0.0156117 ,
               nan,  0.66068132, -0.02932857,  0.00478892,  0.53150442,
        0.63336677,  0.78995164,  0.83427048, -0.00998963,  0.79224646,
        0.48634947,  0.30121264,  0.01079009,  0.4587099 ,  0.13440597,
        0.24306919,         nan,         nan, -0.10215457])

## Jenson Shannon Divergence

In [274]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.6317561457293849

## F1 Score, Precision, Recall

In [275]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [276]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [277]:
d

,Precision,Recall,F1-score
0.100,0.343415,0.328235,0.335653
0.200,0.250291,0.559168,0.345798
0.300,0.128192,0.797508,0.220880
0.010,0.452061,0.255558,0.326525
0.001,0.441202,0.283754,0.345380
